In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models,datasets
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
print("done")

In [ ]:
(x_train,y_train),(x_test,y_test)=datasets.cifar10.load_data()

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0d


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(x_train[0])

In [ ]:
y_train = y_train.flatten()
y_test = y_test.flatten()

In [ ]:
gendata=ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

gendata.fit(x_train)

In [ ]:
y_train[:5]

In [ ]:
# model=models.Sequential()

# model.add(layers.Conv2D(32,(2,2),activation='relu',input_shape=(32,32,3)))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling2D(2,2))

# model.add(layers.Conv2D(16,(2,2),activation='relu'))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling2D(2,2))

# model.add(layers.Conv2D(8,(2,2),activation='relu'))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling2D(2,2))

# model.add(layers.Conv2D(4,(2,2),activation='relu'))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling2D(2,2))

# model.add(layers.Flatten())
# model.add(layers.Dense(128,activation='relu'))
# model.add(layers.Dropout(0.5))
# model.add(layers.Dense(10,activation='softmax'))

In [ ]:
cnn=models.Sequential([
    layers.Conv2D(filters=32,kernel_size=(3,3), padding='same',activation='relu',input_shape=(32,32,3)),
    layers.MaxPooling2D((2,2)),
    layers.BatchNormalization(),

    layers.Conv2D(filters=64,kernel_size=(3,3), padding='same',activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.BatchNormalization(),

    layers.Conv2D(filters=128,kernel_size=(3,3), padding='same',activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.BatchNormalization(),
    
    layers.Conv2D(filters=156,kernel_size=(3,3), padding='same',activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.BatchNormalization(),
    
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(10,activation='softmax')
])

In [ ]:
cnn.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
trained = cnn.fit(
    gendata.flow(x_train, y_train, batch_size=64),
    epochs=20,
    validation_data=(x_test, y_test)
)

In [ ]:
pred=cnn.predict(x_test)

In [ ]:
pred.flatten()
pred[:5]

In [ ]:
predclasses=[]
for i in pred:
    predclasses.append(np.argmax(i))

In [ ]:
predclasses[:5]

In [ ]:
outputdf = pd.DataFrame({
    'predicted values': predclasses,
    'actual values': y_test,
    'success?': predclasses == y_test
})


In [ ]:
outputdf

In [ ]:
outputdf['success?'].value_counts()

In [ ]:
cnn.evaluate(x_test,y_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt 
import seaborn as sns

In [ ]:
print(classification_report(y_test,predclasses))

In [ ]:
cm=tf.math.confusion_matrix(labels=y_test,predictions=predclasses)
sns.heatmap(cm, annot=True,fmt='d')
plt.title("confusion matrix")
plt.xlabel("Predicted values")
plt.ylabel("Actual values")